<a href="https://colab.research.google.com/github/Mithun-033/Text-To-SQL-GPT/blob/main/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
from tokenizers import Tokenizer
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.models import BPE
from dataclasses import dataclass

In [20]:
@dataclass
class Config:
    n_embed:int=1
    cwl:int=1
    b_size:int=1
    head_size :int=1
    n_head :int=1
    vocab_size :int=1
    n_layer :int=1

In [2]:
df = pd.read_json("hf://datasets/Clinton/Text-to-sql-v1/texttosqlv2.jsonl", lines=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [45]:
text=df["instruction"]
struct=df["input"]
struct=pd.DataFrame([i[13:] for i in struct])
query=df["response"]

data=pd.concat([text,struct,query],axis=1).fillna("").agg(" | ".join,axis=1).str.strip()
data="[EOS][BOS]".join(data)
data[:1000]

'Name the home team for carlton away team | table_name_77 (\n    home_team VARCHAR,\n    away_team VARCHAR\n) | SELECT home_team FROM table_name_77 WHERE away_team = "carlton" [EOS] [BOS]what will the population of Asia be when Latin America/Caribbean is 783 (7.5%)? | table_22767 (\n    "Year" real,\n    "World" real,\n    "Asia" text,\n    "Africa" text,\n    "Europe" text,\n    "Latin America/Caribbean" text,\n    "Northern America" text,\n    "Oceania" text\n) | SELECT "Asia" FROM table_22767 WHERE "Latin America/Caribbean" = \'783 (7.5%)\' [EOS] [BOS]How many faculty members do we have for each gender? Draw a bar chart, order by the Y-axis in descending. | Student (\n    StuID INTEGER,\n    LName VARCHAR(12),\n    Fname VARCHAR(12),\n    Age INTEGER,\n    Sex VARCHAR(1),\n    Major INTEGER,\n    Advisor INTEGER,\n    city_code VARCHAR(3)\n)\n\nCREATE TABLE Faculty (\n    FacID INTEGER,\n    Lname VARCHAR(15),\n    Fname VARCHAR(15),\n    Rank VARCHAR(15),\n    Sex VARCHAR(1),\n    

In [ ]:
tok=Tokenizer(BPE)
tok.from_pretrained=ByteLevel(add_prefix_space=True)
trainer=BpeTrainer(vocab_size=Config.vocab_size,special_tokens=["[EOS]","[BOS]"])
tok.train_from_iterator